In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!source myenv/bin/activate

In [3]:
!pip install torchmultimodal-nightly
!pip install datasets
!pip install transformers

  Using cached torchmultimodal_nightly-2023.1.30-py37-none-any.whl (126 kB)
  Using cached iopath-0.1.10-py3-none-any.whl
  Using cached DALL_E-0.1-py3-none-any.whl (6.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Using cached pytest-7.3.1-py3-none-any.whl (320 kB)
  Using cached mypy-1.2.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.8 MB)
  Using cached portalocker-2.7.0-py2.py3-none-any.whl (15 kB)
  Using cached pycryptodomex-3.17-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Using cached lxml-4.9.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (6.6 MB)
  Using cached typed_ast-1.5.4-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (843 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
  Using cached exceptiongroup-1.1.1-py3-none-any.whl (14 kB)
  Using cached iniconfig-2.0.0-py3-none-any.whl (5.

In [39]:
with open("vocabs/vocabulary_100k.txt") as f:
  vocab = f.readlines()

answer_to_idx = {}
for idx, entry in enumerate(vocab):
  answer_to_idx[entry.strip("\n")] = idx
print(len(vocab))
print(vocab[:5])
from datasets import load_dataset
dataset = load_dataset("textvqa")

75501
['`\n', '``\n', '<\n', '=\n', '>\n']


No config specified, defaulting to: textvqa/textvqa
Found cached dataset textvqa (/home/jupyter/.cache/huggingface/datasets/textvqa/textvqa/0.5.1/9b89037cc122c3b495b155a1bce4170851829843454e88f236bb8715d977c027)


  0%|          | 0/3 [00:00<?, ?it/s]

In [44]:
import numpy as np
import torch
from torchvision import transforms
from collections import defaultdict
from transformers import BertTokenizer
from transformers import AlbertTokenizer
from transformers import AutoTokenizer
from transformers import DistilBertTokenizer
from transformers import RobertaTokenizer
from functools import partial
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


True


In [50]:
def transform(tokenizer, input):
  batch = {}
  image_transform = transforms.Compose([transforms.ToTensor(), transforms.Resize([224,224])])
  image = image_transform(input["image"][0].convert("RGB"))
  batch["image"] = [image]

  tokenized=tokenizer(input["question"],return_tensors='pt',padding="max_length",max_length=512)
  batch.update(tokenized)


  ans_to_count = defaultdict(int)
  for ans in input["answers"][0]:
    ans_to_count[ans] += 1
  max_value = max(ans_to_count, key=ans_to_count.get)
  ans_idx = answer_to_idx.get(max_value,0)
  batch["answers"] = torch.as_tensor([ans_idx])
  return batch

# tokenizer=BertTokenizer.from_pretrained("bert-base-uncased",padding="max_length",max_length=512)
# tokenizer = AutoTokenizer.from_pretrained("albert-base-v2", padding="max_length", max_length=512)
tokenizer=DistilBertTokenizer.from_pretrained("distilbert-base-uncased",padding="max_length",max_length=512)
#tokenizer=RobertaTokenizer.from_pretrained("roberta-base",padding="max_length",max_length=512)
transform=partial(transform,tokenizer)
dataset.set_transform(transform)

In [14]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.1 MB/s eta 0:00:0000:0100:01


In [51]:
dataset_partial = dataset["train"].train_test_split(test_size=0.1)

from torch import nn
BATCH_SIZE = 64
from torch.utils.data import DataLoader
from torchmultimodal.models.flava.model import flava_model_for_classification
import pickle

In [ ]:
model = flava_model_for_classification(num_classes=len(vocab))
model = model.to(device)
test_dataloader = DataLoader(dataset_partial["test"], batch_size= 1)
optimizer = torch.optim.AdamW(model.parameters())
total_correct = 0
total_samples = 0
model_out = []
actual_out = []
t = 0
for idx, batch in enumerate(test_dataloader):
    batch["input_ids"] = batch["input_ids"].to(device)
    batch["image"] = batch["image"].to(device)
    zero_tensor = torch.zeros_like(batch["answers"])
    zero_tensor = zero_tensor.to(device) 
    #print(batch["input_ids"].size())
    batch["answers"] = batch["answers"].to(device)
    out = model(text = batch["input_ids"], image = batch["image"], labels = zero_tensor)
    #out = out.to(device)
    #print(out.logits.size())
    predicted_probs = torch.softmax(out.logits, dim = 0)
    #print(predicted_probs.size())
    # Calculate the predicted labels
    predicted_labels = torch.argmax(predicted_probs, dim=1)
    model_out.append(predicted_labels)
    #actual_out.append(batch["answers"])
    actual_out.append(batch["answers"].cpu())
    #print(predicted_labels.size())
    # Update counters
    total_correct += torch.sum(predicted_labels == batch["answers"])
    total_samples += len(batch["answers"])
    print(total_correct, total_samples)

testing_accuracy = float(total_correct) / total_samples
# print(f"Testing accuracy for Flava's Berttokenizer: {testing_accuracy:.4f}")
# print(f"Testing accuracy for Flava's Alberttokenizer: {testing_accuracy:.4f}")
print(f"Testing accuracy for Flava's DistilBerttokenizer: {testing_accuracy:.4f}")

tensor(0, device='cuda:0') 1
tensor(0, device='cuda:0') 2
tensor(1, device='cuda:0') 3
tensor(2, device='cuda:0') 4
tensor(3, device='cuda:0') 5
tensor(3, device='cuda:0') 6
tensor(4, device='cuda:0') 7
tensor(5, device='cuda:0') 8
tensor(6, device='cuda:0') 9
tensor(7, device='cuda:0') 10
tensor(8, device='cuda:0') 11
tensor(8, device='cuda:0') 12
tensor(8, device='cuda:0') 13
tensor(8, device='cuda:0') 14
tensor(8, device='cuda:0') 15
tensor(9, device='cuda:0') 16
tensor(10, device='cuda:0') 17
tensor(11, device='cuda:0') 18
tensor(12, device='cuda:0') 19
tensor(13, device='cuda:0') 20
tensor(14, device='cuda:0') 21
tensor(15, device='cuda:0') 22
tensor(15, device='cuda:0') 23
tensor(15, device='cuda:0') 24
tensor(16, device='cuda:0') 25
tensor(16, device='cuda:0') 26
tensor(17, device='cuda:0') 27
tensor(18, device='cuda:0') 28
tensor(19, device='cuda:0') 29
tensor(20, device='cuda:0') 30
tensor(21, device='cuda:0') 31
tensor(22, device='cuda:0') 32
tensor(23, device='cuda:0') 33
te

In [9]:
from sklearn.metrics import precision_recall_fscore_support

# Concatenate the actual_out and model_out lists
y_true = np.concatenate(actual_out)
#y_pred = np.concatenate([m.detach().cpu().numpy() for m in model_out])
y_pred = np.concatenate([m.detach().cpu().numpy() for m in model_out])

# Calculate the precision, recall, and F1-score
precision, recall, f1_score, support = precision_recall_fscore_support(y_true, y_pred, average='macro', zero_division=1)

# Print the results
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1_score:.4f}")
#from sklearn.metrics import classification_report
#print(classification_report(actual_out, model_out))


Precision: 0.9994
Recall: 0.0012
F1-score: 0.0008
